In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
doc = []
with open("data/dates.txt") as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head()

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
dtype: object

In [5]:
def matcher(x):
    # For later use
    month_mapper = {
        1 : ("Jan", "Jan.", "January", "January."),
        2 : ("Feb", "Feb.", "February", "February."),
        3 : ("Mar", "Mar.", "March", "March."),
        4 : ("Apr", "Apr.", "April", "April."),
        5 : ("May", "May.", "May", "May."),
        6 : ("Jun", "Jun.", "June", "June."),
        7 : ("Jul", "Jul.", "July", "July."),
        8 : ("Aug", "Aug.", "August", "August."),
        9 : ("Sep", "Sep.", "September", "September."),
        10 : ("Oct", "Oct.", "October", "October."),
        11 : ("Nov", "Nov.", "November", "November."),
        12 : ("Dec", "Dec.", "December", "December."),
    }
    ############################## MATCHER 1 ##############################
    try:
        ###### MATCH: 04/20/2009; 04/20/09; 4/20/09; 4/3/09
        match = re.findall(r"[\d]{1,2}[/\-][\d]{1,2}[/\-][\d]{2,4}", x)[0]
        ###### PROCESSING
        if "/" in match:
            items = match.split("/")
        if "-" in match:
            items = match.split("-")

        if len(items[0]) == 1:
            items[0] = "0" + items[0]

        if len(items[1]) == 1:
            items[1] = "0" + items[1]

        if len(items[2]) == 2:
            items[2] = "19" + items[-1]

        processed_match = "/".join(items)
        return processed_match
    except:        
        ############################## MATCHER 2 ##############################
        try:
            ###### MATCH: Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009
            match = re.findall(r"[^\W\d_]{3,}[\-\s\.]{1,2}[\d]{1,2}[\-\s\,]{1,2}[\d]{4}", x)[0]
            ###### PROCESSING
            items = match.replace(",", "").split(" ")

            for i, k in month_mapper.items():
                if items[0] in k:
                    items[0] = str(i)

            if len(items[0]) == 1:
                items[0] = "0" + items[0]

            if len(items[1]) == 1:
                items[1] = "0" + items[1]

            processed_match = "/".join(items)
            return processed_match
        except:
            ############################## MATCHER 3 ##############################
            try:
                ###### MATCH: 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
                match = re.findall(r"[\d]{1,2}\s[^\W\d_]{3,}[\-\s\.]{1,2}[\d]{4}", x)[0]
                ###### PROCESSING
                items = match.replace(",", "").split(" ")

                for i, k in month_mapper.items():
                    if items[1] in k:
                        items[1] = str(i)

                if len(items[0]) == 1:
                    items[0] = "0" + items[0]

                if len(items[1]) == 1:
                    items[1] = "0" + items[1]

                day = items[0]
                month = items[1]

                items[1] = day
                items[0] = month

                processed_match = "/".join(items)
                return processed_match
            except:
                ############################## MATCHER 4 ##############################
                try:
                    ###### MATCH: Feb 2009; Sep 2009; Oct 2010
                    match = re.findall(r"[^\W\d_]{3,}\s[\d]{4}", x)[0]
                    ###### PROCESSING
                    items = match.replace(",", "").split(" ")

                    for i, k in month_mapper.items():
                        if items[0] in k:
                            items[0] = str(i)

                    if len(items[0]) == 1:
                        items[0] = "0" + items[0]

                    items.insert(1, "01")

                    processed_match = "/".join(items)
                    return processed_match
                except:
                    ############################## MATCHER 5 ##############################
                    try:
                        ###### MATCH: 6/2008; 12/2009
                        match = re.findall(r"[\d]{1,2}/[\d]{4}", x)[0]
                        ###### PROCESSING
                        items = match.replace(",", "").split("/")

                        if len(items[0]) == 1:
                            items[0] = "0" + items[0]

                        items.insert(1, "01")

                        processed_match = "/".join(items)
                        return processed_match
                    except:
                        ############################## MATCHER 6 ##############################
                        try:
                            ###### MATCH: 2009; 2010
                            match = re.findall(r"[\d]{4}", x)[0]
                            ###### PROCESSING
                            items = match.replace(",", "").split("/")

                            items.insert(0, "01")
                            items.insert(0, "01")

                            processed_match = "/".join(items)
                            return processed_match
                        except:
                            return np.nan

######################################################################
test = df.apply(lambda x: matcher(x))

print(f"Solved cases: {test.notna().sum()}")
test[test.notna()]

Solved cases: 500


0      03/25/1993
1      06/18/1985
2      07/08/1971
3      09/27/1975
4      02/06/1996
          ...    
495    01/01/1979
496    01/01/2006
497    01/01/2008
498    01/01/2005
499    01/01/1980
Length: 500, dtype: object

In [21]:
# 235, 245
from datetime import datetime

s = "05/01/2005"
datetime.strptime(s, "%m/%d/%Y")

datetime.datetime(2005, 5, 1, 0, 0)

In [22]:
def to_time(x):
    try:
        return datetime.strptime(x, "%m/%d/%Y")
    except:
        return np.nan

In [24]:
test.apply(lambda x: to_time(x)).sort_values()

9     1971-04-10
84    1971-05-18
2     1971-07-08
53    1971-07-11
28    1971-09-12
         ...    
467          NaT
468          NaT
479          NaT
487          NaT
488          NaT
Length: 500, dtype: datetime64[ns]

# Putting everything together

In [33]:
def date_sorter():
    import re
    import numpy as np
    from datetime import datetime

    def matcher(x):
        # For later use
        month_mapper = {
            1 : ("Jan", "Jan.", "January", "January."),
            2 : ("Feb", "Feb.", "February", "February."),
            3 : ("Mar", "Mar.", "March", "March."),
            4 : ("Apr", "Apr.", "April", "April."),
            5 : ("May", "May.", "May", "May."),
            6 : ("Jun", "Jun.", "June", "June."),
            7 : ("Jul", "Jul.", "July", "July."),
            8 : ("Aug", "Aug.", "August", "August."),
            9 : ("Sep", "Sep.", "September", "September."),
            10 : ("Oct", "Oct.", "October", "October."),
            11 : ("Nov", "Nov.", "November", "November."),
            12 : ("Dec", "Dec.", "December", "December."),
        }
        ############################## MATCHER 1 ##############################
        try:
            ###### MATCH: 04/20/2009; 04/20/09; 4/20/09; 4/3/09
            match = re.findall(r"[\d]{1,2}[/\-][\d]{1,2}[/\-][\d]{2,4}", x)[0]
            ###### PROCESSING
            if "/" in match:
                items = match.split("/")
            if "-" in match:
                items = match.split("-")

            if len(items[0]) == 1:
                items[0] = "0" + items[0]

            if len(items[1]) == 1:
                items[1] = "0" + items[1]

            if len(items[2]) == 2:
                items[2] = "19" + items[-1]

            processed_match = "/".join(items)
            return processed_match
        except:        
            ############################## MATCHER 2 ##############################
            try:
                ###### MATCH: Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009
                match = re.findall(r"[^\W\d_]{3,}[\-\s\.]{1,2}[\d]{1,2}[\-\s\,]{1,2}[\d]{4}", x)[0]
                ###### PROCESSING
                items = match.replace(",", "").split(" ")

                for i, k in month_mapper.items():
                    if items[0] in k:
                        items[0] = str(i)

                if len(items[0]) == 1:
                    items[0] = "0" + items[0]

                if len(items[1]) == 1:
                    items[1] = "0" + items[1]

                processed_match = "/".join(items)
                return processed_match
            except:
                ############################## MATCHER 3 ##############################
                try:
                    ###### MATCH: 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
                    match = re.findall(r"[\d]{1,2}\s[^\W\d_]{3,}[\-\s\.]{1,2}[\d]{4}", x)[0]
                    ###### PROCESSING
                    items = match.replace(",", "").split(" ")

                    for i, k in month_mapper.items():
                        if items[1] in k:
                            items[1] = str(i)

                    if len(items[0]) == 1:
                        items[0] = "0" + items[0]

                    if len(items[1]) == 1:
                        items[1] = "0" + items[1]

                    day = items[0]
                    month = items[1]

                    items[1] = day
                    items[0] = month

                    processed_match = "/".join(items)
                    return processed_match
                except:
                    ############################## MATCHER 4 ##############################
                    try:
                        ###### MATCH: Feb 2009; Sep 2009; Oct 2010
                        match = re.findall(r"[^\W\d_]{3,}\s[\d]{4}", x)[0]
                        ###### PROCESSING
                        items = match.replace(",", "").split(" ")

                        for i, k in month_mapper.items():
                            if items[0] in k:
                                items[0] = str(i)

                        if len(items[0]) == 1:
                            items[0] = "0" + items[0]

                        items.insert(1, "01")

                        processed_match = "/".join(items)
                        return processed_match
                    except:
                        ############################## MATCHER 5 ##############################
                        try:
                            ###### MATCH: 6/2008; 12/2009
                            match = re.findall(r"[\d]{1,2}/[\d]{4}", x)[0]
                            ###### PROCESSING
                            items = match.replace(",", "").split("/")

                            if len(items[0]) == 1:
                                items[0] = "0" + items[0]

                            items.insert(1, "01")

                            processed_match = "/".join(items)
                            return processed_match
                        except:
                            ############################## MATCHER 6 ##############################
                            try:
                                ###### MATCH: 2009; 2010
                                match = re.findall(r"[\d]{4}", x)[0]
                                ###### PROCESSING
                                items = match.replace(",", "").split("/")

                                items.insert(0, "01")
                                items.insert(0, "01")

                                processed_match = "/".join(items)
                                return processed_match
                            except:
                                return np.nan

    def to_time(x):
        try:
            return datetime.strptime(x, "%m/%d/%Y")
        except:
            return np.nan

    df2 = df.apply(lambda x: to_time(matcher(x))).sort_values().reset_index()
    return df2["index"]

In [34]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    467
496    468
497    479
498    487
499    488
Name: index, Length: 500, dtype: int64